In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [6]:
from autoplan.trainer import ParserTrainer
from autoplan.dataset import PrelabeledDataset, build_synthetic_dataset
from autoplan.generator import ProgramGenerator
from autoplan.vis import plot_all_accuracy, plot_all_cm
from autoplan.token import PyretTokenizer, OCamlTokenizer

from grammars.rainfall.ocaml import Program
from grammars.rainfall.labels import GeneralRainfallLabels

from tqdm import tqdm_notebook as tqdm
import pandas as pd
import torch
import os

# device = torch.device('cuda:0')
device = torch.device('cpu')
REPO_DIR = os.path.expanduser('~/autoplan')

In [7]:
dataset_name = 'T1'
student_dataset = PrelabeledDataset.load(f'{REPO_DIR}/data/rainfall/{dataset_name}.pkl')

In [9]:
generator = ProgramGenerator(grammar=Program())
program, choices, choice_options, labels = generator.generate()
print(program)

let  rain_helper (aloi : <class 'type'> list) : <class 'type'> list =
    match aloi with
    | [] -> []
    | hd :: _ -> if hd = -999 then []
    | hd :: _ -> if hd < 0 then rain_helper _
        else when hd >= 0 then hd ::  _;;let  rainfall aloi =
    
    if (List.length (rain_helper aloi) = 0) then 0 else
    (List.fold_right (+) rain_helper aloi 0) / List.length (rain_helper aloi);;
    


In [10]:
synth_dataset = build_synthetic_dataset(
    GeneralRainfallLabels,
    N=100,
    tokenizer=OCamlTokenizer(),
    generator=ProgramGenerator(grammar=Program()),
    vocab_index=student_dataset.vocab_index)

Generating programs...


 18%|█▊        | 18/100 [00:00<00:00, 176.74it/s]

Generated 96 unique programs.
Tokenizing programs...


100%|██████████| 100/100 [00:00<00:00, 114.31it/s]

Building dataset metadata...


In [11]:
trainer = ParserTrainer(synth_dataset, device)
losses = []
train_eval = []
val_eval = []

In [14]:
for _ in tqdm(range(20)):
    losses.append(trainer.train_one_epoch())
    train_eval, val_eval = trainer.eval()
    train_evals.append(train_eval)
    val_evals.append(val_eval)

RuntimeError: index out of range at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:193

In [ ]:
pd.Series(losses).plot()

In [ ]:
plot_all_accuracy(train_evals)

In [ ]:
plot_all_accuracy(val_evals)

In [ ]:
plot_all_cm(val_evals[-1])